<a href="https://colab.research.google.com/github/callmidrey/Home_Sales/blob/main/Home_Sales_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import findspark and initialize.
!pip install findspark
import findspark
!which spark-submit
#findspark.init()

In [6]:
# Import packages
!pip install pyspark
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f02961abc905bd27cc28d1db2d6f23efcba71157bc63d770427bf444cfa8898e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",",header=True, inferSchema=True,ignoreLeadingWhiteSpace=True)
data_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# get data types
data_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [11]:
# 2. Create a temporary view of the DataFrame.
data_df.createOrReplaceTempView('home_sales')

In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS Year,
        FORMAT_NUMBER(AVG(price), 2) AS AvgPrice,
        bedrooms
FROM home_sales
WHERE bedrooms = 4
GROUP BY Year, bedrooms
""").show()

+----+----------+--------+
|Year|  AvgPrice|bedrooms|
+----+----------+--------+
|2020|298,353.78|       4|
|2019|300,263.70|       4|
|2022|296,363.88|       4|
|2021|301,819.44|       4|
+----+----------+--------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS Year,
        FORMAT_NUMBER(AVG(price), 2) AS AvgPrice,
        bedrooms,
        bathrooms
FROM home_sales
WHERE bedrooms = 3 AND
          bathrooms = 3
GROUP BY Year, bedrooms,bathrooms
""").show()

+----+----------+--------+---------+
|Year|  AvgPrice|bedrooms|bathrooms|
+----+----------+--------+---------+
|2019|287,287.82|       3|        3|
|2021|294,211.46|       3|        3|
|2022|292,725.69|       3|        3|
|2020|294,204.16|       3|        3|
+----+----------+--------+---------+



In [14]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS Year,
        FORMAT_NUMBER(AVG(price), 2) AS AvgPrice,
        sqft_living AS SquareFeet,
        bedrooms,
        bathrooms,
        floors
FROM home_sales
WHERE bedrooms = 3 AND
      bathrooms = 3 AND
      floors = 2
GROUP BY Year, bedrooms, bathrooms, floors,sqft_living
HAVING SquareFeet >= 2000
""").show()

+----+----------+----------+--------+---------+------+
|Year|  AvgPrice|SquareFeet|bedrooms|bathrooms|floors|
+----+----------+----------+--------+---------+------+
|2021|342,842.00|      2040|       3|        3|     2|
|2019|344,395.50|      2225|       3|        3|     2|
|2020|328,475.00|      2430|       3|        3|     2|
|2019|238,168.50|      2146|       3|        3|     2|
|2021|223,046.00|      2016|       3|        3|     2|
|2021|326,515.00|      2258|       3|        3|     2|
|2019|277,687.00|      2104|       3|        3|     2|
|2020|241,470.00|      2218|       3|        3|     2|
|2019|200,041.00|      2189|       3|        3|     2|
|2021|178,946.33|      2116|       3|        3|     2|
|2020|139,421.00|      2002|       3|        3|     2|
|2021|172,883.00|      2167|       3|        3|     2|
|2019|161,976.00|      2411|       3|        3|     2|
|2019|380,939.00|      2154|       3|        3|     2|
|2021|341,846.00|      2317|       3|        3|     2|
|2020|314,

In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
    SELECT view,
           FORMAT_NUMBER(AVG(price),2) AS AvgPrice
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|    AvgPrice|
+----+------------+
|  85|1,056,336.74|
|  65|  736,679.93|
|  53|  755,214.80|
|  78|1,080,649.37|
|  81|1,053,472.79|
|  76|1,058,802.78|
|  91|1,137,372.73|
|  93|1,026,006.06|
|  52|  733,780.26|
|  86|1,070,444.25|
|  94|1,033,536.20|
|  57|  734,340.50|
|  54|  798,684.82|
|  96|1,017,815.92|
|  92|  970,402.55|
|  64|  767,036.67|
|  61|  746,877.59|
|  88|1,031,719.35|
|  72|  780,914.67|
|  59|  791,453.00|
+----+------------+
only showing top 20 rows

--- 0.8789215087890625 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
spark.sql("""
    SELECT view,
           FORMAT_NUMBER(AVG(price),2) AS AvgPrice
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|    AvgPrice|
+----+------------+
|  85|1,056,336.74|
|  65|  736,679.93|
|  53|  755,214.80|
|  78|1,080,649.37|
|  81|1,053,472.79|
|  76|1,058,802.78|
|  91|1,137,372.73|
|  93|1,026,006.06|
|  52|  733,780.26|
|  86|1,070,444.25|
|  94|1,033,536.20|
|  57|  734,340.50|
|  54|  798,684.82|
|  96|1,017,815.92|
|  92|  970,402.55|
|  64|  767,036.67|
|  61|  746,877.59|
|  88|1,031,719.35|
|  72|  780,914.67|
|  59|  791,453.00|
+----+------------+
only showing top 20 rows

--- 0.5305171012878418 seconds ---


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
data_df.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")
#data_df.head()

In [22]:
# 11. Read the formatted parquet data.
par_df = spark.read.parquet('home_partitioned')

In [23]:
# 12. Create a temporary table for the parquet data.
par_df.createOrReplaceTempView('par_home')

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
    SELECT view,
           FORMAT_NUMBER(AVG(price),2) AS AvgPrice
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|    AvgPrice|
+----+------------+
|  85|1,056,336.74|
|  65|  736,679.93|
|  53|  755,214.80|
|  78|1,080,649.37|
|  81|1,053,472.79|
|  76|1,058,802.78|
|  91|1,137,372.73|
|  93|1,026,006.06|
|  52|  733,780.26|
|  86|1,070,444.25|
|  94|1,033,536.20|
|  57|  734,340.50|
|  54|  798,684.82|
|  96|1,017,815.92|
|  92|  970,402.55|
|  64|  767,036.67|
|  61|  746,877.59|
|  88|1,031,719.35|
|  72|  780,914.67|
|  59|  791,453.00|
+----+------------+
only showing top 20 rows

--- 0.45511770248413086 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False